# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import f1_score, classification_report
from sklearn.datasets import  fetch_california_housing

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e  39.0  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e  24.0  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e  58.0   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e  33.0  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e  16.0  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0          0.0        0.0           0.0     0.0     0.0    Maham Ofracculy   
1        109.0        9.0          25.0   549.0    44.0       Juanna Vines   
2         43.0     3576.0           0.0  6715.0    49.0      Altark Susent   
3          0.0     1283.0         371.0  3329.0   193.0       Solam Susent   
4        303.0       70.0         151.0   565.0     2.0  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [3]:
spaceship.dropna(inplace=True)
spaceship["Cabin"] = spaceship["Cabin"].apply(lambda x: x.split("/")[0])
spaceship = spaceship.drop(["PassengerId", "Name"], axis=1)
spaceship.head()

HomePlanet CryoSleep Cabin  Destination   Age    VIP  RoomService  \
0     Europa     False     B  TRAPPIST-1e  39.0  False          0.0   
1      Earth     False     F  TRAPPIST-1e  24.0  False        109.0   
2     Europa     False     A  TRAPPIST-1e  58.0   True         43.0   
3     Europa     False     A  TRAPPIST-1e  33.0  False          0.0   
4      Earth     False     F  TRAPPIST-1e  16.0  False        303.0   

   FoodCourt  ShoppingMall     Spa  VRDeck  Transported  
0        0.0           0.0     0.0     0.0        False  
1        9.0          25.0   549.0    44.0         True  
2     3576.0           0.0  6715.0    49.0        False  
3     1283.0         371.0  3329.0   193.0        False  
4       70.0         151.0   565.0     2.0         True

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [4]:
#your code here
X = spaceship.drop("Transported", axis=1)
y = spaceship['Transported']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lasso = LassoCV(cv=5)

lasso.fit(X_train, y_train)

importance = pd.Series(lasso.coef_, index=X.columns)
print(importance[importance != 0])

Age            -0.000610
RoomService    -0.000192
FoodCourt       0.000043
ShoppingMall    0.000028
Spa            -0.000100
VRDeck         -0.000089
dtype: float64


In [5]:
X = spaceship.drop("Transported", axis=1)
y = spaceship['Transported']

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")

Tamaño del conjunto de entrenamiento: 5284 muestras
Tamaño del conjunto de prueba: 1322 muestras


In [6]:
normalizer = MinMaxScaler()

normalizer.fit(X_train)

MinMaxScaler()

In [7]:
X_train_norm = normalizer.transform(X_train)

X_test_norm = normalizer.transform(X_test)

In [8]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [9]:
#your code here
X = spaceship.drop("Transported", axis=1)
y = spaceship['Transported']
X = pd.get_dummies(X, drop_first=True)

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Hacer predicciones
y_pred_rf = rf_model.predict(X_test)


- Evaluate your model

In [10]:
#your code here
rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f'Accuracy del modelo de Random Forest: {rf_accuracy:.2f}')

Accuracy del modelo de Random Forest: 0.81


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [11]:
#your code here
grid = {"n_estimators": [50, 100],
        "estimator__max_leaf_nodes": [250, None],
        "estimator__max_depth": [10, 30]
}

- Run Grid Search

In [12]:
ada_reg = AdaBoostClassifier(DecisionTreeClassifier(), algorithm='SAMME')
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)
model.fit(X_train_norm, y_train)


GridSearchCV(cv=5,
             estimator=AdaBoostClassifier(algorithm='SAMME',
                                          estimator=DecisionTreeClassifier()),
             param_grid={'estimator__max_depth': [10, 30],
                         'estimator__max_leaf_nodes': [250, None],
                         'n_estimators': [50, 100]})

- Evaluate your model

In [13]:
model.best_params_

{'estimator__max_depth': 30,
 'estimator__max_leaf_nodes': 250,
 'n_estimators': 100}

In [14]:
best_model = model.best_estimator_

In [15]:
pred = best_model.predict(X_test_norm)

# Calcular métricas de clasificación
print("Accuracy:", accuracy_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred, average='weighted'))
print("Classification Report:\n", classification_report(y_test, pred))

Accuracy: 0.7972768532526475
F1 Score: 0.7972824210784838
Classification Report:
               precision    recall  f1-score   support

       False       0.79      0.80      0.80       653
        True       0.81      0.79      0.80       669

    accuracy                           0.80      1322
   macro avg       0.80      0.80      0.80      1322
weighted avg       0.80      0.80      0.80      1322

